**13 June 2016**

For yeast, files under directories `short-single-dx` where x = 50, 100, 200 created with commands:
```
$ PREFIX=/global/scratch/seanla/Data/yeast
$ OUTPUTDIR=$PREFIX/short-single-d200
$ OUTPUT=$OUTPUTDIR/yeast-short-single-d200
$ cd /home/seanla/Software/art_bin_GreatSmokyMountains/
$ mkdir -p $OUTPUTDIR
$ ./art_illumina -s -sam -i $PREFIX/saccharomyces-cerevisiae chromosome1_sequence.fasta -l 100 -f x -o $OUTPUT
```
where x = 50, 100, 200.

For E.coli, files under directories `short-single-dx` where x = 50, 100, 200 created with commands:
```
$ PREFIX=/global/scratch/seanla/Data/ecoli
$ OUTPUTDIR=$PREFIX/short-single-d200
$ OUTPUT=$OUTPUTDIR/ecoli-short-single-d200
$ cd /home/seanla/Software/art_bin_GreatSmokyMountains/
$ mkdir -p $OUTPUTDIR
$ ./art_illumina -s -sam -i $PREFIX/escherichia-coli_reference.fasta -l 100 -f x -o $OUTPUT
```
where x = 50, 100, 200



**16 June 2016**

Simulated long read files for E.coli with coverage of 10, 20, 50 and 75 were generated by SimLoRD using a shell/PBS script with the commands

```
PREFIX=/global/scratch/seanla/Data/ecoli
PATH=/home/seanla/Projects/hcstats
REF=$PREFIX/escherichia-coli_reference.fasta
FASTQ=$PREFIX/SRR1284073.fastq


COVERAGE=x
READS=$(/global/software/python/python272/bin/python $PATH/reads4coverage.py -c $COVERAGE -i /global/scratch/seanla/Data/ecoli/SRR1284073.fastq -r /global/scratch/seanla/Data/ecoli/escherichia-coli_reference.fasta)
OUTPUT="$PREFIX/simlord/long-d$COVERAGE/ecoli-long-d$COVERAGE"

/bin/mkdir -p $OUTPUT

echo "Coverage: $COVERAGE"
echo "Reads: $READS"

/home/seanla/Software/anaconda2/envs/simlord/bin/simlord -n $READS -sf $FASTQ -rr $REF $OUTPUT
```

Where `x` was set to 10, 20, 50 and 75 respectively.

Simulated long read files for yeast with coverage of 10, 20, 50 and 75 were generated by SimLoRD using a shell/PBS script with the following commands:

```
# Paths to executables
python=/global/software/python/python272/bin/python
reads4coverage=/home/seanla/Projects/hcstats/reads4coverage.py
simlord=/home/seanla/Software/anaconda2/envs/simlord/bin/simlord

# Paths to data
data=/global/scratch/seanla/Data/yeast
ref=$data/saccharomyces-cerevisiae-chromosome1_sequence.fasta
fastq=$data/SRR1284662.fastq

# Perform simlord for coverage 10, 20, 50, 75

# ------------------ Coverage x -----------------------------
# Input parameters for simlord
coverage=x
# Find number of reads required for coverage
reads=$($python $reads4coverage -c $coverage -i $fastq -r $ref)
# Output directory path
output="$data/simlord/long-d$coverage/ecoli-long-d$coverage"

mkdir -p $output

echo "Coverage: $coverage"
echo "Reads: $reads"

$simlord -n $reads -sf $fastq -rr $ref $output
```
where `x` is 10, 20, 50 and 75.

** 20 June 2016 **

Paired-end short reads of coverage 50, 100 and 200 for ecoli and yeast were generated with the following commands:

```
prefix=/global/scratch/seanla/Data/ecoli
outputdir=$prefix/art/short-paired-d200
output=$outputdir/ecoli-short-paired-d200
ref=$prefix/escherichia-coli_reference.fasta
mkdir -p $outputdir
cd /home/seanla/Software/art_bin_GreatSmokyMountains/
./art_illumina -p -i $ref -l 100 -f x -m 300 -s 25 -o $output
```

```
prefix=/global/scratch/seanla/Data/yeast
outputdir=$prefix/art/short-paired-d100
output=$outputdir/yeast-short-paired-d100
ref=$prefix/accharomyces-cerevisiae-chromosome1_sequence.fasta
mkdir -p $outputdir
cd /home/seanla/Software/art_bin_GreatSmokyMountains/
./art_illumina -p -i $ref -l 100 -f x -m 300 -s 25 -o $output
```

Where the `-l 100` indicates a short read length of 100, `-f x` indicates a fold coverage of `x` (of which `x` was set to 50, 100 and 200), `-m 300` indicates a mean fragment length of 300 and `-s 25` indicates a standard deviation of fragment length of 25.

**21 June 2016**
LoRDeC, Jabba and proovread were performed on short reads of coverage 50, 100 and 200 and long reads of coverage 10, 20, 50 and 75.

For LoRDeC, the following parameters were used:
- kmer length of 19
- error rate of 0.4
- number of target k-mers: 5
- max branch exploration of 200

For Jabba, the following parameters were used:
- The de Bruijn graph was created with Brownie with a k-mer length of 45 (graphCorrection mode)
- Thread usage of 6
- Minimum size of MEM of 20
- k-mer size of 45

For proovread, preprocessing was performed as follows:
- bwa-proovread was used to map the short reads onto the long reads
- the resulting SAM file was sorted using samtools
- For E.coli runs 50Sx50L, 50Sx75L, 100Sx50L, 100Sx75L, 200Sx10L, 200Sx20L, 200Sx50L, 200Sx75L, the parameter "long read quality offset value" could not be automatically determined and so the corrections failed. In future runs this value must be inputted manually.


### Monday, 27 June 2016 ###

LoRDeC, Jabba and proovread were used to correct long reads of coverage 10, 20, 50 and 75 using short reads of coverage 50, 100 and 200. Details of the pipeline are as follows:

#### LoRDeC ####
To run LoRDeC, the following command was used:

```
./lordec-correct -T ${PBS_NUM_PPN} --trials 5 --branch 200 --errorrate 0.4 -2 $short1 $short2 -k 19 -s 3 -i $long -o $output
```

An explanation of the parameters used are as follows:
- `--trials` is the number of paths to try from a k-mer
- `--branch` is the number of branches to explore
- `--errorrate` is the expected maximum error rate of the long reads
- `-k 19` is the k-mer length, which is set to 19
- `-s 3` is the solid k-mer abundance threshold

8 threads were allocated to each job.

#### Jabba ####
Short reads were first corrected with Karect with the following command:

```
karect -correct -inputfile=$short1 -inputfile=$short2 -resultdir=$output -tempdir=$output -celltype=haploid -matchtype=hamming -threads=${PBS_NUM_PPN}
```

The parameters are as follows:
- Since we are working with E.coli and yeast genomes, the celltype was set to haploid.
- We used the simple matching type of hamming.

Next, we constructed the de Bruijn graph using the program Brownie using the following command:

```
brownie graphCorrection -k 75 -p $output $short1 $short2
```

Where k is the kmer length and was set to 75.

Lastly, we corrected long reads using Jabba with the following command:

```
jabba -t ${PBS_NUM_PPN} -l 20 -k 75 -o $output -g $dbgraph -fastq $long
```

Where the parameters `l` and `k` are the minimum seed suze and kmer size, respectively. Notice that the kmer size of Jabba and Brownie are the same.
8 threads were allocated to each job.

#### proovread ####
Before running proovread, short reads were mapped onto long reads using BWA-proovread, a special version of the conventional BWA mapper built for proovread.

First, we indexed the long reads with the command:

```
bwa-proovread index $long
```

Then we mapped the short reads onto the long reads with the command:

```
bwa-proovread mem $long $short1 $short2 > $samOutput
```

Then the outputted SAM file was sorted using SAM tools and converted into binary:

```
samtools sort -T $tempPath -o $bamOutput $samOutput
```

Finally, the long reads were corrected with proovread using the following command:

```
proovread -t ${PBS_NUM_PPN} --lr-qv-offset 70 --bam $bamOutput -l $long -p $output
```

The parameter `--lr-qv-offset 70` indicates that the expected long read phred quality offset is manually set to 70. Normally, proovread automatically detects the quality offset of the long reads, but in datasets where the quality of the long reads are between 64 and 73 this fails so the quality offset must be set manually. The proovread developer indicated through correspondence on Github that this quality offsets for long reads are actually not needed for long reads, so we set the offset to an arbitrary value of 70. 8 threads were allocated to each job.

## Wesdnesday, 6 July 2016 ##

#### CoLoRMap ####
CoLoRMap was used to correct long reads of coverage 10, 20, 50 and 75 using short reads of coverage 50, 100 and 200 with commands similar to:

```bash
$colormap $long $mergedShort $outputPrefix ${PBS_NUM_PPN}
```

The script `runBoth.sh` calls both the `runCorr.sh` and the `runOEA.sh` script to perform regular corrections and corrections with one-end anchors, respectively. 8 threads were allocated to each job.